In [13]:
pip install dask

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
pip install netcdf4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
pip install xarray

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import xarray as xr
import pandas as pd

In [18]:
# ini rangenya diganti sesuai sama tahun tugas sendiri sendiri 
dates_2023 = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

In [19]:
# ganti bagian tahun /2023/ jadi /tahun bagian sendiri/
base_url = "https://data.cci.ceda.ac.uk/thredds/dodsC/esacci/soil_moisture/data/daily_files/COMBINED/v09.1/2023/"
# gausah diubah udah nyesuaiin sendiri
filename_template = "ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-{date_str}000000-fv09.1.nc"

In [20]:
list_of_urls = []

In [21]:
for date in dates_2023:
    # Format tanggalnya jadi string 'YYYYMMDD' (misal: '20230101')
    date_string = date.strftime('%Y%m%d')
    filename = filename_template.format(date_str=date_string)
    list_of_urls.append(base_url + filename)

In [22]:
# buat ngecek url apa aja yang keambil
print("List URL yang dibuat:")
print(list_of_urls[0:3])

List URL yang dibuat:
['https://data.cci.ceda.ac.uk/thredds/dodsC/esacci/soil_moisture/data/daily_files/COMBINED/v09.1/2023/ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-20230101000000-fv09.1.nc', 'https://data.cci.ceda.ac.uk/thredds/dodsC/esacci/soil_moisture/data/daily_files/COMBINED/v09.1/2023/ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-20230102000000-fv09.1.nc', 'https://data.cci.ceda.ac.uk/thredds/dodsC/esacci/soil_moisture/data/daily_files/COMBINED/v09.1/2023/ESACCI-SOILMOISTURE-L3S-SSMV-COMBINED-20230103000000-fv09.1.nc']


In [23]:
print("Mulai membuka dataset... ")
ds_series = xr.open_mfdataset(
    list_of_urls,
    combine='by_coords',
    #compat='override', #jangan diapa apain, dihapus gpp sih
    join='override',
    parallel=False # jangan diganti True nanti parallel computing ga semua bisa
)

print("Dataset berhasil dimuat:")
print(ds_series)

Mulai membuka dataset... 
Dataset berhasil dimuat:
<xarray.Dataset> Size: 11GB
Dimensions:         (time: 365, lat: 720, lon: 1440)
Coordinates:
  * time            (time) datetime64[ns] 3kB 2023-01-01 ... 2023-12-31
  * lat             (lat) float32 3kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * lon             (lon) float32 6kB -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
Data variables:
    sm              (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sm_uncertainty  (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    flag            (time, lat, lon) float32 2GB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    freqbandID      (time, lat, lon) int16 757MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    dnflag          (time, lat, lon) int8 378MB dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mode            (time, lat, lon) int8 378MB dask.array<chunksize=(1, 720, 1440), meta=n

In [24]:
# ini buat batasan lokasi Indonesia
lat_batas_indo = slice(6, -11)
lon_batas_indo = slice(95, 141)
# ini bikin variable sm_indonesia yang isinya 1 tahun numerical data
sm_indonesia2023 = ds_series['sm'].sel(lat=lat_batas_indo, lon=lon_batas_indo)

In [25]:
df_sm2023 = sm_indonesia2023.to_dataframe().reset_index()
df_sm2023   

,time,lat,lon,sm
0,2023-01-01,5.875,95.125,NaN
1,2023-01-01,5.875,95.375,NaN
2,2023-01-01,5.875,95.625,NaN
3,2023-01-01,5.875,95.875,NaN
4,2023-01-01,5.875,96.125,NaN
...,...,...,...,...
4566875,2023-12-31,-10.875,139.875,NaN
4566876,2023-12-31,-10.875,140.125,NaN
4566877,2023-12-31,-10.875,140.375,NaN
4566878,2023-12-31,-10.875,140.625,NaN


In [27]:
df_avg = df_sm2023.groupby('time')['sm'].mean()

In [28]:
df_avg_final = df_avg.to_frame().reset_index()
df_avg_final

,time,sm
0,2023-01-01,0.333221
1,2023-01-02,0.330340
2,2023-01-03,0.333150
3,2023-01-04,0.334386
4,2023-01-05,0.331193
...,...,...
360,2023-12-27,0.334275
361,2023-12-28,0.338789
362,2023-12-29,0.335544
363,2023-12-30,0.336394


In [29]:
df_avg_final.to_csv("Soil_Moisture_Indonesia_RATA-RATA_2023.csv", index=False)

In [ ]:
# ganti namanya sesuai tahun
# df_sm2023.to_csv('Soil Moisture Indonesia 2023.csv', index=False)